### 0. Reflection 的重點說明

#### 功能：旅遊行程規劃助理

**流程說明：**
1. 使用者輸入旅遊需求（地點、天數、旅遊型態、預算）
2. `model_writer` (夢想旅程編織者) 生成第一版行程初稿（風格有創意、融合四大元素）
3. `model_reviewer` (資深在地導遊) 檢查行程的時間可行性、參觀深度、行程密度，並提供具體修改建議
4. `model_writer` (夢想旅程編織者) 根據建議產出第二版（優化後的行程）
5. Gradio 呈現：三個欄位：第一版、建議、第二版

#### 1. 讀入金鑰

請依你使用的服務, 決定讀入哪個金鑰

In [16]:
import os
from google.colab import userdata

In [17]:
#【使用 Groq】
api_key = userdata.get('Groq')
os.environ['GROQ_API_KEY']=api_key
provider = "groq"
model = "openai/gpt-oss-120b"

In [18]:
!pip install aisuite[all]

### 2. 基本設定

In [19]:
import aisuite as ai

In [20]:
provider_writer = "groq"
model_writer="openai/gpt-oss-120b"

provider_reviewer = "groq"
model_reviewer = "openai/gpt-oss-120b"

#provider_reviewer = "openai"
#model_reviewer = "gpt-4o"

標準回應函式

In [21]:
def reply(system="請用台灣習慣的中文回覆。",
          prompt="hi",
          provider="groq",
          model="openai/gpt-oss-120b"
          ):

    client = ai.Client()

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]


    response = client.chat.completions.create(model=f"{provider}:{model}", messages=messages)

    return response.choices[0].message.content

####  3. 設定「旅程編織者」和「在地導遊」

In [22]:
system_writer = """你是一位名為「夢想旅程編織者」的頂級旅遊作家，熱衷於發掘獨特的體驗和隱藏的秘境。你的任務是根據用戶的需求，設計一份極富創意和吸引力的分日旅遊行程初稿。

【用戶輸入】將包含：地點、天數、旅遊型態（緊湊/享受自由）、大致預算。

【規劃原則】
1. 風格融合： 行程中必須平衡地融入以下四大元素：當地特色美食、文化或歷史體驗、戶外活動、以及自然景觀。
2. 輸出格式： 必須使用 Markdown 格式，清楚列出每日的行程，包含景點名稱和簡短介紹。
3. 初稿特性： 在此階段，請先不考慮實際的交通時間和轉乘複雜性，以最大化行程的吸引力。你的目標是打造一份「夢想的骨架」。"""

system_reviewer = """你是一位名為「資深在地導遊兼時間管理大師」的行程審核專家。
你的風格極度嚴謹務實，專注於確保行程的「時間可行性」和「旅遊品質」，
認為旅遊品質優先於景點數量。你的任務是審核「夢想旅程編織者」提供的初稿，並提出具體且強烈的修正建議。

【審核流程】你必須分兩階段思考：

[第一階段：時間與可行性分析]
1. 交通與距離： 評估每日景點之間的地理位置和交通方式，判斷移動時間是否合理，是否安排了太多跨區移動。
2. 參觀時間： 判斷行程為重要景點（如大型博物館、複雜的文化遺址）預留的時間是否足夠。若不足，應指出這是「走馬看花」且不符品質的安排。
3. 密度判斷： 每天安排的活動數量是否過多，造成明顯的疲憊感或「軍事化旅遊」傾向。

[第二階段：修正建議]
針對所有不可行或影響品質之處，提出具體的、可執行的修改建議。例如：「建議將 Day 3 的 A 景點與
Day 2 的 B 景點對調，以優化 Day 2 的路線，並為 A 景點多爭取 1 小時參觀時間。」

【最終輸出】
你必須直接輸出修正建議，不須解釋你的思維過程，但你的建議必須基於現實可行性。"""

In [23]:
def reflect_post(prompt):
    # Step 1: Writer 初稿
    first_version = reply(system_writer, prompt,
                          provider=provider_writer,
                          model=model_writer
                          )

    # Step 2: Reviewer 給建議
    suggestion = reply(system_reviewer, first_version,
                       provider=provider_reviewer,
                       model=model_reviewer
                       )

    # Step 3: Writer 再寫一次（根據建議）
    second_prompt = f"""這是我剛設計的行程初稿：
        {first_version}
        這是我收到的修改建議：
        {suggestion}
        你已收到「資深在地導遊兼時間管理大師」針對你的初稿行程所提供的具體修改建議。

        現在你的最終任務是：根據這些建議，**大幅調整**初稿中不合理的時間、
        距離和景點安排，將其修正為一份**專業且具備高度可行性**的最終版行程。
        """

    second_version = reply(system_writer, second_prompt,
                           provider=provider_writer,
                           model=model_reviewer
                           )

    return first_version, suggestion, second_version

### 4. 用 Gradio 打造的對話機器人 Web App!

In [24]:
!pip install gradio

In [25]:
import gradio as gr

In [26]:
with gr.Blocks() as demo:
    gr.Markdown("### ✈️ 旅遊行程規劃助理 (Reflection 模式)")
    user_input = gr.Textbox(label="請輸入你的旅遊需求", placeholder="例如：地點：宜蘭, 天數：三天兩夜, 旅遊型態：享受自由, 預算：每人 5000 台幣")
    btn = gr.Button("生成行程 & 取得專業建議")

    with gr.Row():
        out1 = gr.Textbox(
            label="🌟 第一版行程 (夢想旅程編織者)",
            lines=10,
            max_lines = 40
        )
        out2 = gr.Textbox(
            label="🧐 專業修改建議 (資深在地導遊)",
            lines=5,
            max_lines=40
            )
        out3 = gr.Textbox(
            label="✨ 最終可行行程 (優化版)",
            lines=40,
            max_lines=40
            )

    btn.click(reflect_post, inputs=[user_input], outputs=[out1, out2, out3])

In [ ]:
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://364835682b9b6b1b1f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
